# threading

In [ ]:
open rust_operators

In [ ]:
//// test

open testing

## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::thread::JoinHandle<$0>\")>]\n#endif\ntype std_thread_JoinHandle<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::Arc<$0>\")>]\n#endif\ntype std_sync_Arc<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::Mutex<$0>\")>]\n#endif\ntype std_sync_Mutex<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::MutexGuard<$0>\")>]\n#endif\ntype std_sync_MutexGuard<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::PoisonError<$0>\")>]\n#endif\ntype std_sync_PoisonError<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::mpsc::Receiver<$0>\")>]\n#endif\ntype std_sync_mpsc_Receiver<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::mpsc::SendError<$0>\")>]\n#endif\ntype std_sync_mpsc_SendError<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::sync::mpsc::Sender<$0>\")>]\n#endif\ntype std_sync_mpsc_Sender<'T> = class end"

## join_handle

In [ ]:
nominal join_handle t = $'std_thread_JoinHandle<`t>'

## spawn

In [ ]:
inl spawn forall t. depth flag (x : () -> t) : join_handle t =
    if flag = 1u8
    then (!\($'"true; let __result = std::thread::spawn(move || { //"') : bool) |> ignore
    else (!\($'"true; let __result = std::thread::spawn(|| { //"') : bool) |> ignore

    let x' = x ()
    inl x' = join x'

    x' |> rust.fix_closure depth

    !\($'"__result"')

## join'

In [ ]:
inl join' forall t.
    (x : join_handle t)
    : resultm.result'
        t
        (
            rust.box (
                rust.lifetime_join'
                    rust.dyn'
                    (
                        rust.lifetime_join
                            rust.any
                            (
                                rust.lifetime_join'
                                    rust.send
                                    rust.static_lifetime
                            )
                    )
            )
        ) =
    !\\(x, $'"std::thread::JoinHandle::join($0)"')

## cancellation_token

In [ ]:
nominal cancellation_token = $'System.Threading.CancellationToken'

## cancellation_token_source

In [ ]:
nominal cancellation_token_source = $'System.Threading.CancellationTokenSource'

## cancellation_token_registration

In [ ]:
nominal cancellation_token_registration = $'System.Threading.CancellationTokenRegistration'

## cancellation_source_token

In [ ]:
inl cancellation_source_token (x : cancellation_token_source) : cancellation_token =
    $'!x.Token'

## cancellation_source_cancel

In [ ]:
inl cancellation_source_cancel (x : cancellation_token_source) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!x.Cancel' ()
        | _ => fun () => null ()

## create_linked_token_source

In [ ]:
inl create_linked_token_source (x : array_base cancellation_token) : cancellation_token_source =
    x |> $'System.Threading.CancellationTokenSource.CreateLinkedTokenSource'

## sleep

In [ ]:
inl sleep (duration : date_time.duration) : () =
    inl duration = join duration
    !\($'"std::thread::sleep(!duration)"')

## concurrent_stack

In [ ]:
nominal concurrent_stack t = $'System.Collections.Concurrent.ConcurrentStack<`t>'

## concurrent_stack_push

In [ ]:
inl concurrent_stack_push forall t. (item : t) (stack : concurrent_stack t) : () =
    $'!stack.Push' item

## token_none

In [ ]:
inl token_none () : cancellation_token =
    $'`cancellation_token.None'

## new_concurrent_stack

In [ ]:
inl new_concurrent_stack forall t. () : concurrent_stack t =
    $'System.Collections.Concurrent.ConcurrentStack<`t>' ()

## token_register

In [ ]:
inl token_register (fn : () -> ()) (ct : cancellation_token) : cancellation_token_registration =
    fn |> $'!ct.Register'

## new_cancellation_token_source

In [ ]:
inl new_cancellation_token_source () : cancellation_token_source =
    $'new `cancellation_token_source ()'

In [ ]:
inl token_cancellation_requested (ct : cancellation_token) : bool =
    $'!ct.IsCancellationRequested'

## new_disposable_token

In [ ]:
inl new_disposable_token (merge_token : optionm'.option' cancellation_token) =
    run_target function
        | Fsharp (Native) => fun () =>
            inl cts = new_cancellation_token_source ()
            inl cts =
                match merge_token |> optionm'.unbox with
                | None => cts
                | Some merge_token =>
                    create_linked_token_source ;[ cts |> cancellation_source_token; merge_token ]
            inl disposable : _ () = new_disposable fun () =>
                cts |> cancellation_source_cancel
            cts |> cancellation_source_token, disposable
        | _ => fun () => null ()

In [ ]:
//// test

inl run fn =
    inl token, disposable = new_disposable_token (None |> optionm'.box)
    disposable |> use |> ignore
    fn token
    fun () =>
        fn token
    |> async.new_async
    |> async.start

fun () =>
    inl counter = mut 0i32

    inl fn (token : cancellation_token) =
        counter <- *counter + (if token |> token_cancellation_requested then 10 else 1)

    join run fn
    async.sleep 10 |> async.do
    return *counter
|> async.new_async_unit
|> async.run_synchronously
|> _assert_eq 11i32

assert_eq / actual: 11 / expected: 11


## arc

In [ ]:
nominal arc t = $'std_sync_Arc<`t>'

## new_arc

In [ ]:
inl new_arc forall t. (x : t) : arc t =
    !\\(x, $'"std::sync::Arc::new($0)"')

## mutex

In [ ]:
nominal mutex t = $'std_sync_Mutex<`t>'

## new_mutex

In [ ]:
inl new_mutex forall t. (x : t) : mutex t =
    !\\(x, $'"std::sync::Mutex::new($0)"')

## new_arc_mutex

In [ ]:
inl new_arc_mutex forall t. (x : t) : arc (mutex t) =
    x |> new_mutex |> new_arc

## arc_clone

In [ ]:
inl arc_clone forall t. (x : arc t) : arc t =
    inl x = join x
    !\($'"std::sync::Arc::clone(&!x)"')

## mutex_guard

In [ ]:
nominal mutex_guard t = $'std_sync_MutexGuard<`t>'

## poison_error

In [ ]:
nominal poison_error t = $'std_sync_PoisonError<`t>'

## arc_mutex_lock

In [ ]:
inl arc_mutex_lock forall t. (x : arc (mutex t)) : resultm.result' (mutex_guard t) (poison_error (mutex_guard t)) =
    inl x = join x
    !\($'"!x.lock()"')

## mutex_guard_ref

In [ ]:
inl mutex_guard_ref forall t. (x : mutex_guard t) : rust.ref' t =
    !\\(x, $'"&$0"')

## mutex_guard_ref_mut

In [ ]:
inl mutex_guard_ref_mut forall t. (x : mutex_guard t) : rust.ref' (rust.mut' t) =
    (!\($'"true; let mut !x = !x"') : bool) |> ignore
    !\\(x, $'"&mut $0"')

## mutex_guard_as_mut

In [ ]:
inl mutex_guard_as_mut forall (t : * -> *) u. (x : mutex_guard (t u)) : t (rust.ref' (rust.mut' u)) =
    (!\($'"true; let mut !x = !x"') : bool) |> ignore
    !\\(x, $'"$0.as_mut()"')

## channel_receiver

In [ ]:
nominal channel_receiver t = $'std_sync_mpsc_Receiver<`t>'

## channel_sender

In [ ]:
nominal channel_sender t = $'std_sync_mpsc_Sender<`t>'

## new_channel

In [ ]:
inl new_channel () : channel_sender sm'.std_string * arc (channel_receiver sm'.std_string) =
    !\($'"{ let (sender, receiver) = std::sync::mpsc::channel(); (sender, std::sync::Arc::new(receiver)) }"')

## send_error

In [ ]:
nominal send_error t = $'std_sync_mpsc_SendError<`t>'

## channel_send

In [ ]:
inl channel_send forall t. (line : t) (sender : rust.ref' (channel_sender t)) : resultm.result' () (send_error sm'.std_string) =
    !\\((sender, line), $'"$0.send($1)"')

## main

In [ ]:
inl main () =
    types ()
    $'let new_disposable_token x = !new_disposable_token x' : ()